In [1]:
!pip install tensorflow==2.1.0rc

  Using cached https://files.pythonhosted.org/packages/1c/6d/7aae38a9022f982cf8167775c7fc299f203417b698c27080ce09060bba07/google_auth-1.11.0-py2.py3-none-any.whl
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
  Found existing installation: google-auth 1.4.0
    Uninstalling google-auth-1.4.0:
      Successfully uninstalled google-auth-1.4.0


In [2]:
!pip install grpcio==1.24.3

In [3]:
!pip install google-auth==1.4.0

  Using cached https://files.pythonhosted.org/packages/a4/d4/b1adfcf5c83cb9c52be33f147b72c47877619dac68efd2707ed98fea9331/google_auth-1.4.0-py2.py3-none-any.whl
ERROR: tensorboard 2.0.2 has requirement google-auth<2,>=1.6.3, but you'll have google-auth 1.4.0 which is incompatible.
ERROR: google-api-python-client 1.7.11 has requirement google-auth>=1.4.1, but you'll have google-auth 1.4.0 which is incompatible.
ERROR: earthengine-api 0.1.209 has requirement google-auth>=1.4.1, but you'll have google-auth 1.4.0 which is incompatible.
  Found existing installation: google-auth 1.11.0
    Uninstalling google-auth-1.11.0:
      Successfully uninstalled google-auth-1.11.0


In [0]:
import tensorflow as tf

In [0]:
def Model(input_shape,output_shape):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense((256*256*3),use_bias = False , input_shape = input_shape))
  model.add(tf.keras.layers.LeakyReLU())
  model.add(tf.keras.layers.MaxPool2D())
  model.add(tf.keras.layers.Reshape((256,256,3)))
  model.add(tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
  model.add(tf.keras.layers.LeakyReLU())
  model.add(tf.keras.layers.MaxPool2D())
  model.add(tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False ,activation = 'sigmoid'))
  model.add(tf.keras.layers.Reshape(output_shape))
  return model

In [0]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def loss(output,mask):
  loss = cross_entropy(mask,output)
  return loss

In [0]:
from zipfile import ZipFile

In [0]:
with ZipFile('/content/sky.zip','r') as zipObj:
  zipObj.extractall()

In [0]:
import os
import cv2
images = os.listdir('/content/sky/data')
opt = os.listdir('/content/sky/groundtruth')
def Gen(i):
  img = cv2.imread('/content/sky/data/' + images[i])
  img = (img / 127.5) - 1.0
  opt_img = cv2.imread('/content/sky/groundtruth/' + opt[i])
  opt_img = ((opt_img[:,:,0]+opt_img[:,:,1]+opt_img[:,:,2]) / (255.0 *3))
  return img , opt_img

In [0]:
EPOCHS = 50
optimizer = tf.keras.optimizers.Adam(1e-4)
def train(model , epochs):
  e_loss = 0
  for e in range(epochs):
    print(e_loss)
    for i in range(len(images)):
      print(i)
      t,o = Gen(i)
      with tf.GradientTape() as tape:
        o_m = model(t)
        loss = loss(o_m , m)
        if i == 0:
          e_loss = loss
        else:
          e_loss += loss
        grads = tape.gradient(loss,model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [11]:
t,o = Gen(0)
Skyfinder = Model(t.shape,o.shape)
Skyfinder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 401, 600, 196608)  589824    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 401, 600, 196608)  0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 200, 300, 196608)  0         
_________________________________________________________________
reshape (Reshape)            (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 256, 256, 128)     9600      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256, 256, 128)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 128)     0

In [0]:
train(Skyfinder,EPOCHS)

0

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

